In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sort import Sort

# Load the pre-trained TensorFlow model
model = tf.saved_model.load("/content/ssd_mobilenet_v2_coco/saved_model")

# Create SORT tracker
tracker = Sort()

# Upload the video file and the sort.py file to Colab
from google.colab import files

# Upload the video file
uploaded = files.upload()

# Extract the uploaded files
!unzip sort.zip

# Replace 'your_video_filename.mp4' with the actual file name
video_path = "/content/your_video_filename.mp4"

# Open a video capture
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the image for object detection
    input_tensor = tf.convert_to_tensor([frame])
    detections = model(input_tensor)

    # Process the detections
    boxes = []
    for detection in detections['detection_boxes'][0].numpy():
        ymin, xmin, ymax, xmax = detection
        xmin, xmax, ymin, ymax = int(xmin * frame.shape[1]), int(xmax * frame.shape[1]), \
                                 int(ymin * frame.shape[0]), int(ymax * frame.shape[0])
        boxes.append([xmin, ymin, xmax - xmin, ymax - ymin])

    # Apply SORT tracker for object tracking
    trackers = tracker.update(np.array(boxes))
    for i, new_box in enumerate(trackers):
        x, y, w, h, track_id = map(int, new_box)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"Person {int(track_id)}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Intelligent Camera Decision-Making', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()